<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Inception_Network__Pytorch_Implementation_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on Google CoLaB with Pytorch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t:float = 129.3)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) /60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>>formating the time\tplease wait...\n>>>time elapsed: \t{time_fmt()}")


Mounted at /content/drive
You are on Google CoLaB with Pytorch version: 1.8.1+cu101
>>>>formating the time	please wait...
>>>time elapsed: 	0: 02: 09.00


In [55]:
#In this notbook we are going to imliment the Inception network from scratch in Pytorch:
#We adjust to suit for CIFAR10 data (incase you want to train on it for demo)


In [56]:
import torch
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import time, datetime


In [57]:
#Convolutional block: This class will compute all convolutions required for our network

In [78]:
class CNNBLOCK(nn.Module):
  def __init__(self,in_channels,out_channels,**kwargs):
    super(CNNBLOCK, self).__init__()
    self.relu = nn.ReLU()
    self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, **kwargs)
    self.batchnorm = nn.BatchNorm2d(num_features = out_channels)
  def forward(self, x):
    x = self.relu(self.conv(x))
    return self.batchnorm(x)


In [79]:
#Inception block: This class will compose an inception block: This is simply combinations 
#of sevral convolutions according to INCEPTION ARCHITECTURE 

In [80]:
class InceptionBlock(nn.Module):
  def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, pool_1x1):
    super(InceptionBlock, self).__init__()
    self.block_1 = CNNBLOCK(in_channels = in_channels, out_channels = out_1x1, kernel_size = 1)
    self.block_2 = nn.Sequential(CNNBLOCK(in_channels = in_channels, out_channels = red_3x3, kernel_size = 1),
                                 CNNBLOCK(in_channels = red_3x3, out_channels = out_3x3, kernel_size = 3, stride = 1, padding = 1))
    self.block_3 = nn.Sequential(CNNBLOCK(in_channels = in_channels, out_channels = red_5x5, kernel_size = 1),
                                 CNNBLOCK(in_channels = red_5x5, out_channels = out_5x5, kernel_size = 5, stride = 1, padding = 2))
    self.block_4 = nn.Sequential(nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1),
                                 CNNBLOCK(in_channels = in_channels, out_channels = pool_1x1, kernel_size = 1))
  
  def forward(self, input_tensor):
    '''concatenate along the filter's dimension = [batch x filter x H x W]'''
    x = torch.cat([self.block_1(input_tensor), self.block_2(input_tensor), self.block_3(input_tensor),self.block_4(input_tensor)], 1)
    return x


In [81]:
#Model class: In this class we build our model using the above defined classes:

In [82]:
class GooGleNet(nn.Module):
  def __init__(self, num_classes = 10, in_channels = 3):
    super(GooGleNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 64, kernel_size = 7, padding = 3, stride = 2)
    self.maxpool1 = nn.MaxPool2d(kernel_size = 3, padding = 1, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 192, kernel_size = 3, padding = 1, stride = 1)
    self.maxpool2 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.inception_1 = InceptionBlock(192, 64,96,128,16,32,32) #as indicated from inception paper
    self.inception_2 = InceptionBlock(256, 128,128,192,32,96,64)
    self.maxpool3 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.inception_3 = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
    self.inception_4 = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
    self.inception_5 = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
    self.inception_6 = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
    self.inception_7 = InceptionBlock(528, 256, 160, 320, 32, 128,128)
    self.maxpool4 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.inception_8 = InceptionBlock(832, 256, 160, 320, 32,128, 128)
    self.inception_9 = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
    self.flt = nn.Flatten()
    self.dropout = nn.Dropout(p = 0.4)
    self.fc = nn.Linear(in_features = 1024, out_features = num_classes)
  
  def forward(self, input_tensor):
    m = nn.Upsample(size = 7) #To suit cifar10 data
    x = m(input_tensor)
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.inception_1(x)
    x = self.inception_2(x)
    x = self.maxpool3(x)
    x = self.inception_3(x)
    x = self.inception_4(x)
    x = self.inception_5(x)
    x = self.inception_6(x)
    x = self.inception_7(x)
    x = self.maxpool4(x)
    x = self.inception_8(x)
    x = self.inception_9(x)
    x = self.flt(x)
    x = x.reshape(x.shape[0],-1)
    x = self.dropout(x)
    x = self.fc(x)
    return x


In [83]:
#Instantiate the model class and check the output shape if it is of desired shape: 
#(batch_size = 64,num_classes = 10(cifar10 data) )

In [84]:
rnd_data = torch.randn(64, 3, 32,32)
model = GooGleNet()
print(f"output shape: {model(rnd_data).shape}")

output shape: torch.Size([64, 10])


In [85]:
print(model)

GooGleNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception_1): InceptionBlock(
    (block_1): CNNBLOCK(
      (relu): ReLU()
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (block_2): Sequential(
      (0): CNNBLOCK(
        (relu): ReLU()
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
        (batchnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): CNNBLOCK(
        (relu): ReLU()
        (conv): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batchnorm): BatchNorm2d(1